<a href="https://colab.research.google.com/github/ashlin07/ieee-ospi-atm-intrustion-detection-system/blob/atm_instrusion_model/ATM_intrustion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import numpy as np

In [3]:
NUM_CLASSES = 4

In [ ]:
dataset = 'coords_new.csv'
X_dataset = np.genfromtxt(dataset, delimiter=',', dtype='float32', usecols=range(1, (33* 4) + 1))

y_dataset = np.genfromtxt(dataset, delimiter=',', dtype='int32',usecols=0)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

In [ ]:
print(y_dataset[2000:2005])

[1 1 1 1 1]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

In [ ]:
!pip install tensorflow

In [ ]:
model = Sequential(
    [
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=100
)


Epoch 1/100
138/138 [==============================] - 1s 2ms/step - loss: 1.1042
Epoch 2/100
138/138 [==============================] - 0s 3ms/step - loss: 0.5612
Epoch 3/100
138/138 [==============================] - 0s 3ms/step - loss: 0.3196
Epoch 4/100
138/138 [==============================] - 0s 2ms/step - loss: 0.2071
Epoch 5/100
138/138 [==============================] - 0s 2ms/step - loss: 0.1480
Epoch 6/100
138/138 [==============================] - 0s 3ms/step - loss: 0.1084
Epoch 7/100
138/138 [==============================] - 0s 3ms/step - loss: 0.0868
Epoch 8/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0720
Epoch 9/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0571
Epoch 10/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0510
Epoch 11/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0435
Epoch 12/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0391
Epoch 13/100


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (32, 25)                  3325      
                                                                 
 dense_1 (Dense)             (32, 15)                  390       
                                                                 
 dense_2 (Dense)             (32, 4)                   64        
                                                                 
Total params: 3,779
Trainable params: 3,779
Non-trainable params: 0
_________________________________________________________________


In [ ]:
p_preferred = model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

138/138 [==============================] - 0s 2ms/step
two example output vectors:
 [[  9.999396     0.19094057  -6.5719757  -24.374111  ]
 [  7.5522738   -3.2511322  -18.015587    -7.7787814 ]]
largest value 30.87763 smallest value -65.52314


In [ ]:
print(y_train[100])
print( f"{p_preferred[100]}, category: {np.argmax(p_preferred[100])}")

2
[-11.536453   -3.0765965   8.584723  -11.0381975], category: 2


In [ ]:
count=0
for i in range(3000):
    m, n = X_dataset.shape
    random_index = np.random.randint(m)
    prediction = model.predict(X_dataset[random_index].reshape(1,33*(4)),verbose=None)
    prediction_p = tf.nn.softmax(prediction)
    yhat = np.argmax(prediction_p)
    if(yhat!=y_dataset[random_index]):
        count+=1

print(count)

KeyboardInterrupt: ignored

In [ ]:
m, n = X_dataset.shape
random_index = np.random.randint(m)
prediction = model.predict(X_dataset[random_index].reshape(1,33*(4)))
prediction_p = tf.nn.softmax(prediction)
yhat = np.argmax(prediction_p)

In [ ]:
print( f"{display_errors(model,X,y)} errors out of {len(X)} ")

In [ ]:
model.save('model_intrusion.h5')
